# 1. Analysis of Given Features

We analyzed the correlation level of features from given data ('tr_eikon_eod_data.csv').

Conclusion: Upon examining the provided data ('tr_eikon_eod_data.csv'), we found weak correlations (magnitude < 0.04) between the given features and stock direction. To improve prediction accuracy, we'll need to incorporate additional or newly created features.

In [1]:
import pandas as pd
import numpy as np

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20024\2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data = pd.read_csv('./data/tr_eikon_eod_data.csv', index_col = 0, parse_dates = True)
data = data[['AAPL.O', 'MSFT.O', 'INTC.O', 'AMZN.O', 'GS.N', 'SPY']]
data.dropna(inplace = True)

# Direction: 1 means that tomorrow's price is >= today's price and -1 means that tomorrow's price < today's price
data['direction'] = np.where(data['AAPL.O'] <= data['AAPL.O'].shift(-1), 1, -1)

data

,AAPL.O,MSFT.O,INTC.O,AMZN.O,GS.N,SPY,direction
Date,,,,,,,
2010-01-04,30.572827,30.950,20.88,133.90,173.08,113.33,1
2010-01-05,30.625684,30.960,20.87,134.69,176.14,113.63,-1
2010-01-06,30.138541,30.770,20.80,132.25,174.26,113.71,-1
2010-01-07,30.082827,30.452,20.60,130.00,177.67,114.19,1
2010-01-08,30.282827,30.660,20.83,133.52,174.31,114.57,-1
...,...,...,...,...,...,...,...
2018-06-25,182.170000,98.390,50.71,1663.15,221.54,271.00,1
2018-06-26,184.430000,99.080,49.67,1691.09,221.58,271.60,-1
2018-06-27,184.160000,97.540,48.76,1660.51,220.18,269.35,1


In [3]:
# Correlation Analysis for each feature
data.corr()['direction']

AAPL.O      -0.036055
MSFT.O      -0.016139
INTC.O      -0.021609
AMZN.O      -0.021744
GS.N        -0.009771
SPY         -0.021940
direction    1.000000
Name: direction, dtype: float64

# 2. Additional Features: Feature Collection from Online Data

In [ ]:
import requests
import pandas as pd

In [44]:
API_KEY_ALPHA_VANTAGE = "ICAG0XBGXUELT5PD"

In [53]:
def add_features(input_file, api_urls, api_paths):
    df = pd.read_csv(input_file, index_col=0, parse_dates=True)
    for (url, paths) in zip(api_urls, api_paths):
        r = requests.get(url)
        data = r.json()
        print(data)
        for path in paths:
            data = data[path]
        new_df = pd.DataFrame(data).transpose()
        new_df.index = pd.to_datetime(new_df.index)
        df = pd.merge(df, new_df, left_index=True, right_index=True, how='left')
    return df

input_file = 'microsoft_output.csv'
output_file = 'microsoft_output.csv'
stock_name = 'MSFT'
api_urls = [
    f'https://www.alphavantage.co/query?function=STOCH&symbol={stock_name}&interval=daily&apikey={API_KEY_ALPHA_VANTAGE}',
    f'https://www.alphavantage.co/query?function=RSI&symbol={stock_name}&interval=daily&time_period=10&series_type=open&apikey={API_KEY_ALPHA_VANTAGE}',
    f'https://www.alphavantage.co/query?function=ADX&symbol={stock_name}&interval=daily&time_period=10&apikey={API_KEY_ALPHA_VANTAGE}',
    f'https://www.alphavantage.co/query?function=CCI&symbol={stock_name}&interval=daily&time_period=10&apikey={API_KEY_ALPHA_VANTAGE}',
    f'https://www.alphavantage.co/query?function=AROON&symbol={stock_name}&interval=daily&time_period=14&apikey={API_KEY_ALPHA_VANTAGE}',
    f'https://www.alphavantage.co/query?function=OBV&symbol={stock_name}&interval=daily&apikey={API_KEY_ALPHA_VANTAGE}',
    f'https://www.alphavantage.co/query?function=AD&symbol={stock_name}&interval=daily&apikey={API_KEY_ALPHA_VANTAGE}',
    f'https://www.alphavantage.co/query?function=SMA&symbol={stock_name}&interval=daily&time_period=10&series_type=open&apikey={API_KEY_ALPHA_VANTAGE}',
    f'https://www.alphavantage.co/query?function=EMA&symbol={stock_name}&interval=daily&time_period=10&series_type=open&apikey={API_KEY_ALPHA_VANTAGE}'
    ]
api_paths = [
    ['Technical Analysis: STOCH'],
    ['Technical Analysis: RSI'],
    ['Technical Analysis: ADX'],
    ['Technical Analysis: CCI'],
    ['Technical Analysis: AROON'],
    ['Technical Analysis: OBV'],
    ['Technical Analysis: Chaikin A/D'],
    ['Technical Analysis: SMA'],
    ['Technical Analysis: EMA']
    ]

df = add_features(input_file, api_urls, api_paths)
df

/var/folders/ql/hnw16fjd1bb7227zpj_cw9140000gn/T/ipykernel_91866/1270173916.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(input_file, index_col=0, parse_dates=True)


{'Meta Data': {'1: Symbol': 'MSTF', '2: Indicator': 'Stochastic (STOCH)', '3: Last Refreshed': '2024-03-28', '4: Interval': 'daily', '5.1: FastK Period': 5, '5.2: SlowK Period': 3, '5.3: SlowK MA Type': 0, '5.4: SlowD Period': 3, '5.5: SlowD MA Type': 0, '6: Time Zone': 'US/Eastern Time'}, 'Technical Analysis: STOCH': {'2024-03-28': {'SlowK': '0.0000', 'SlowD': '0.0000'}, '2024-03-27': {'SlowK': '0.0000', 'SlowD': '0.0000'}, '2024-03-26': {'SlowK': '0.0000', 'SlowD': '0.0000'}, '2024-03-25': {'SlowK': '0.0000', 'SlowD': '0.0000'}, '2024-03-22': {'SlowK': '0.0000', 'SlowD': '0.0000'}, '2024-03-21': {'SlowK': '0.0000', 'SlowD': '0.0000'}, '2024-03-20': {'SlowK': '0.0000', 'SlowD': '0.0000'}, '2024-03-19': {'SlowK': '0.0000', 'SlowD': '0.0000'}, '2024-03-18': {'SlowK': '0.0000', 'SlowD': '0.0000'}, '2024-03-15': {'SlowK': '0.0000', 'SlowD': '0.0000'}, '2024-03-14': {'SlowK': '0.0000', 'SlowD': '0.0000'}, '2024-03-13': {'SlowK': '0.0000', 'SlowD': '0.0000'}, '2024-03-12': {'SlowK': '0.0000

,MSFT.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA
Date,,,,,,,,,,,,
2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-01,30.950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05-01,30.960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-01,30.770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-01,30.452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,98.390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-06-26,99.080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-06-27,97.540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df.to_csv(output_file, index=True)

# 3. Additional Features: Feature Creation

In [25]:
SMA_short = 7
SMA_long = 20

def create_features(input_file, stock_name, output_name):
    df = pd.read_csv(input_file, index_col=0, parse_dates=True).dropna()   

    #create SMA_EMA_ratio, SMA_Short, SMA_Long and SMA_Long_Short feature
    df['SMA_EMA_ratio'] = df['SMA'] / df['EMA']
    df['SMA_Short'] = df[stock_name].rolling(SMA_short).mean()
    df['SMA_Long'] = df[stock_name].rolling(SMA_long).mean()
    df['SMA_Long_Short'] = df['SMA_Short'] / df['SMA_Long']
    
    df.to_csv(output_name, index=True)

file_name = './data/apple.csv'
stock_name = 'AAPL.O'
output_file = './data/apple.csv'

create_features(file_name, stock_name, output_file)